In [ ]:
# Here, we use the developed tool to generate images of the heatmaps associated with the paper's results section

In [ ]:
# jaimie: check whether you need all these imports
import importlib
import numpy as np
import math as m
import statistics as st
import cmath
import matplotlib.pyplot as plt 
import itertools
import random
from operator import add
from setup_nx import *
from graphviz import Source, render
import pydot

import datetime
import time

import setup_nx # your own module, setup.nx.py
importlib.reload(setup_nx)
import my_feeder_funcs as ff
import my_impedance_funcs as imp
import my_configVis_funcs as vis
import my_detControlMatExistence_funcs as ctrl
import my_detLznRange_funcs as lzn
import my_heatmapSetup_funcs as hm

In [ ]:
# [Essential] specify input feeder data

# Enter the path/name of the impedance model data (excel file)
filepath = "123NF/"
modelpath = filepath + "004_GB_IEEE123_OPAL.xls"
loadfolder = "123NF/"
# load data is not actually used
loadpath = loadfolder + '004_123NF_PVpen100_nocloud_minutewise_whead.csv'
headerpath = '123NF/004_GB_IEEE123_time_header.csv'
load_data = loadpath

#file_name = string specifying name of dot file created when make_graph() is called
file_name = '123NF'
#file_name = 'demo123NF.dot'

# Specify substation kV, kVA bases, name, and the number of timesteps in the load data'
Vbase_ll = 4160
Vbase = Vbase_ll / np.sqrt(3)
Sbase = 5000/3
substation_name = 'bus_150'
timesteps = 1

# initialize some variables
ts = time.time()
print(datetime.datetime.fromtimestamp(ts))
plot = 0 #turn plot on/off
depths = {}
leaves = []

In [ ]:
# [ESSENTIAL] create feeder object

fin_feeder = ff.feeder_init(modelpath,loadfolder,loadpath,timesteps,Vbase_ll,Sbase,depths,leaves)
print("Finished initializing feeder")
ff.make_graph(fin_feeder, file_name)
node_index_map = hm.createNodeIndexMap(fin_feeder) #node indices for indicMat and F matrix
R,X=hm.createRXmatrices_3ph(fin_feeder, node_index_map,depths)

print('depths=',depths) # check this is populated, lists how far each node is from substation
# print list of buses in network
count = 0 
for i in fin_feeder.network:    
    print(i) 
    count += 1
    if count >= 10:
        break
    
import csv  # jaimie do you need this?
graph = fin_feeder.network 

Source.from_file(file_name)
#^ plot network graph inline



In [ ]:

#Source.from_file('colorMap_fig2_PL0001') # display graph in noteboo

#printpng('results_figs/github_figs/123NF_blank.png')

In [ ]:
# Motivating example configuration 2 (same configuration as CPP process except for node 60)
all_act_locs=['bus_60','bus_55','bus_33','bus_5','bus_96','bus_113','bus_197']
# ---------- mark actuator config -------------
vis.markActuatorConfig(all_act_locs,fin_feeder, file_name)
(graph,) = pydot.graph_from_dot_file('actConfig_'+ file_name) # requires 'import pydot'
graph.write_png('config_60.png')
#printpng('results_figs/github_figs/config_60.png')

# Motivating example configuration 1 (same configuration as CPP process except for node 60)
all_act_locs=['bus_61','bus_55','bus_33','bus_5','bus_96','bus_113','bus_197']
# ---------- mark actuator config -------------
vis.markActuatorConfig(all_act_locs,fin_feeder, file_name)
(graph,) = pydot.graph_from_dot_file('actConfig_'+ file_name) # requires 'import pydot'
graph.write_png('config_61.png')
#printpng('results_figs/github_figs/config_61.png')

In [ ]:
# Neighborhood scenario step 7 heatmap, img is of whole feeder

# heatmap/collect stable configs for 7th step only:
set_act_locs = ['bus_82','bus_87','bus_76','bus_49','bus_41','bus_46']
set_perf = ['bus_77','bus_77','bus_77','bus_44','bus_44','bus_44']
addon_locs = ['bus_60']
addon_perf_nodes = ['bus_66']

t = time.time()
# set_act_locs and set_perf are existing pairs, addon_locs and addon_perf_nodes are pairs to add after produce heatmap
lst_feas_configs, lzn_error_run_sum, heatMapNames = hm.runHeatMapProcess(fin_feeder, set_act_locs, set_perf, addon_locs, addon_perf_nodes, node_index_map, substation_name, depths, file_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)
# lst_feas_configs is list of dictionaries
elapsed = time.time() - t
print('Time elapsed=',elapsed)
# takes about an hour to run

(graph,) = pydot.graph_from_dot_file('NPP_heatmap_step0_' + file_name) # requires 'import pydot'
graph.write_png('RHP_nbhd_wideblue.png')
#printpng('results_figs/github_figs/RHP_nbhd_wideblue.png')

In [ ]:
# Neighborhood scenario step 7 heatmap, image is after cropped and neighborhoods 1 and 2 marked
#printpng('results_figs/github_figs/RHP_nbhd_wideblue_cropped_markup.png')

In [ ]:
# Neighborhood scenario branch analysis
# Code to produce data for Table I

# The NPP on the neighborhood configuration for steps 1 through 7 takes a long time to run (evaluating about 800 configurations), so instead
# load data 'lst_feas_configs' from .pkl file
import pickle
filename='nbhs_RHP_lstfeasconfigs.pkl'
with open(filename,'rb') as f:  # Python 3: open(..., 'rb')
    lst_feas_configs = pickle.load(f)
    
# convert lst_feas_configs (list of dictionaries) to newlst_feas_configs (a list of lists)
# input a list of feasible actuator configurations where each configuration is its own list (ie a list of lists)"
length=len(lst_feas_configs)
newlst_feas_configs = []
numact_lst=[] # store number of actuators in separate list
i=0
for dic in lst_feas_configs:
    #print('dic=',dic)
    #print(type(dic))
    i+=1
    if i<length-1:
        #print('dic[act]=',str(dic['act']))
        newlst_feas_configs += [dic['act']]
        numact_lst+=[dic['numfeas']]
        
# create 'placed' variable
# placed is an ordered list of lists, where the k'th ele of the list is the actuators placed at that step
set_act_locs = []
set_perf = []
addon_locs = ['bus_82','bus_87','bus_76','bus_49','bus_41','bus_46','bus_66']
addon_perf_nodes = ['bus_77','bus_77','bus_77','bus_44','bus_44','bus_44','bus_66']

acts=set_act_locs + addon_locs
placed=[]
k=0
while k<=len(acts):
    placed+=[acts[0:k]]
    k+=1
print(placed)

# ----- run assign_network_branches ---------"
branches = vis.assign_network_branches(fin_feeder, substation_name)
vis.mark_network_branches(fin_feeder, branches, file_name, substation_name,depths)
Source.from_file('branch_key:' + file_name)
print('branches table:') # format into a table with index and branch nodes
i=0
for br in branches:
    print(i,',',br)
    i+=1

# ----- run find_good_branches ---------"
# the 'best' branch is the branch with the most actuators from feas configs on it across all configurations
num_good_branches = 8 # return the n best branches
vis.find_good_branches(newlst_feas_configs, branches, num_good_branches, placed)


In [ ]:
# non-auto CPP Process heatmap, Motivating example

# these vars arent used
set_act_locs = ['bus_55','bus_33','bus_5','bus_96','bus_113','bus_197']
set_perf = ['bus_55','bus_33','bus_5','bus_96','bus_113','bus_197']
addon_locs = ['bus_60']
addon_perf_nodes = ['bus_60']

# ------------ run placeMaxColocActs_stopAtInfeas -----------
# place colocated actuators until an infeasible loc is tested, then call find_good_colocated and return 
# produces colormap associated with "motex" motivating example
t = time.time()
max_act_config_stopInfeas=hm.placeMaxColocActs_stopAtInfeas(fin_feeder, file_name, node_index_map, depths, substation_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)
elapsed = time.time() - t
print('Time elapsed=',elapsed)

#printpng('results_figs/github_figs/placeMaxColoc_stopInfeas.png')
# takes about 5 hours
heatmap_colocated_123NF

(graph,) = pydot.graph_from_dot_file('actConfig_'+ file_name) # requires 'import pydot'
graph.write_png('config_61.png')

In [ ]:
# Code to produce data for Table II and III
# Auto-CPP, for random seeds 3,4,5,6,7,8
# ------ Place place_max_colocated_acts -------------
t = time.time()
seed=1
print('----------- Seed='+str(seed)+' ----------------------------')
max_act_config = hm.place_max_coloc_acts(seed,fin_feeder, file_name, node_index_map, depths, substation_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)
# seed=4
seed=2
print('----------- Seed='+str(seed)+' ----------------------------')
max_act_config = hm.place_max_coloc_acts(seed,fin_feeder, file_name, node_index_map, depths, substation_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)
# seed=4
seed=3
print('----------- Seed='+str(seed)+' ----------------------------')
max_act_config = hm.place_max_coloc_acts(seed,fin_feeder, file_name, node_index_map, depths, substation_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)
# seed=4
# print('----------- Seed='+str(seed)+' ----------------------------')
# max_act_config = hm.place_max_coloc_acts(seed,fin_feeder, file_name, node_index_map, depths, substation_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)
# seed=5
# print('----------- Seed='+str(seed)+' ----------------------------')
# max_act_config = hm.place_max_coloc_acts(seed,fin_feeder, file_name, node_index_map, depths, substation_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)
# seed=6
# print('----------- Seed='+str(seed)+' ----------------------------')
# max_act_config = hm.place_max_coloc_acts(seed,fin_feeder, file_name, node_index_map, depths, substation_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)
# seed=7
# print('----------- Seed='+str(seed)+' ----------------------------')
# max_act_config = hm.place_max_coloc_acts(seed,fin_feeder, file_name, node_index_map, depths, substation_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)
# seed=8
# print('----------- Seed='+str(seed)+' ----------------------------')
# max_act_config = hm.place_max_coloc_acts(seed,fin_feeder, file_name, node_index_map, depths, substation_name,Vbase_ll, Sbase, load_data, headerpath, modelpath)
elapsed = time.time() - t
print('Time elapsed=',elapsed)

#printpng('results_figs/github_figs/actConfig_max_coloc_seed3.png')
#printpng('results_figs/github_figs/actConfig_max_coloc_seed4.png')
#printpng('results_figs/github_figs/actConfig_max_coloc_seed5.png')
#printpng('results_figs/github_figs/actConfig_max_coloc_seed6.png')
#printpng('results_figs/github_figs/actConfig_max_coloc_seed7.png')
#printpng('results_figs/github_figs/actConfig_max_coloc_seed8.png')